In [18]:
from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import time
import dlib
import cv2
from gaze_tracking import GazeTracking

EYE_ASPECT_RATIO_THRESHOLD = 0.3
EYE_ASPECT_RATIO_CONSEC_FRAMES = 50
COUNTER = 0
face_cascade = cv2.CascadeClassifier("haarcascades/haarcascade_frontalface_default.xml")

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A+B) / (2*C)
    return ear

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

video_capture = cv2.VideoCapture(0)
pos="center"

t1 = time.monotonic()
gaze = GazeTracking()
cnt=0
while(True):
    ret, frame = video_capture.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gaze.refresh(frame)
    frame = gaze.annotated_frame()
    faces = detector(gray, 0)

    face_rectangle = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in face_rectangle:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    for face in faces:

        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEyeAspectRatio = eye_aspect_ratio(leftEye)
        rightEyeAspectRatio = eye_aspect_ratio(rightEye)
        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2

       
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        
        a=gaze.pupil_left_coords()
        b=gaze.pupil_right_coords()
        
        if(a is None or b is None):
            cv2.putText(frame, "pupil not detected", (150,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2) 
            cnt = 0
            continue        
        
        if cnt ==  10:
            prev_a = a
            prev_b = b
            
        elif cnt == 20:
            pos = "start"
            
        ##initial frames ignored to consider set up time
        elif cnt>20:
            if(prev_a[0]<a[0] and prev_b[0]<b[0]):
                pos="right"

            elif(prev_a[0]>a[0] and prev_b[0]>b[0]):
                pos="left"
            prev_a= a
            prev_b= b
                          
        cv2.putText(frame, pos, (150,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        cv2.circle(frame, a, 3, (0, 255, 0), -1)
        cv2.circle(frame, b, 3, (0, 255, 0), -1)

    cnt+=1
    cv2.imshow('Video', frame)

    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
video_capture.release()
cv2.destroyAllWindows()
